<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 

Follow instructions step by step until the end and submit your complete notebook as an archive (tar -cf groupXnotebook.tar DL_lab2/).

Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by May 29th 2019 (23:59:59 CET).

# Introduction

In the last Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%.  Can  you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to  more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288430956
Epoch:  02   =====> Loss= 0.732285627
Epoch:  03   =====> Loss= 0.600207632
Epoch:  04   =====> Loss= 0.536525863
Epoch:  05   =====> Loss= 0.497833625
Epoch:  06   =====> Loss= 0.471296848
Epoch:  07   =====> Loss= 0.451226831
Epoch:  08   =====> Loss= 0.435932527
Epoch:  09   =====> Loss= 0.423315858
Epoch:  10   =====> Loss= 0.413183023
Epoch:  11   =====> Loss= 0.404133027
Epoch:  12   =====> Loss= 0.396702312
Epoch:  13   =====> Loss= 0.390097731
Epoch:  14   =====> Loss= 0.384495500
Epoch:  15   =====> Loss= 0.379028125
Epoch:  16   =====> Loss= 0.374415280
Epoch:  17   =====> Loss= 0.370363047
Epoch:  18   =====> Loss= 0.366507495
Epoch:  19   =====> Loss= 0.362890023
Epoch:  20   =====> Loss= 0.359804173
Epoch:  21   =====> Loss= 0.356594868
Epoch:  22   =====> Loss= 0.353948794
Epoch:  23   =====> Loss= 0.351219322
Epoch:  24   =====> Loss= 0.348531387
Epoch:  25   =====> Loss= 0.346512246
Epoch:  26   =====> Loss= 0.344198551
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png" width="800" height="600" align="center">





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [3]:
# Functions for weigths and bias initilization 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)

In [4]:
def conv2d(x, W, b, strides=1, padding_ = 'SAME'):  #helper function to define convolution layers
    x = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding=padding_)
    #x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [5]:

def LeNet5_Model(image):    
    # your inmplementation goes here
    shape = image.shape
    #preprocess image : size = (?, 784) to size (?, 28, 28, 1) 
    
    
    weights = {
    # Convolution Layers
    'Wc1': weight_variable((5,5,1,6)), 
    'Wc2': weight_variable((5,5,6,16)) ,
    
    # Dense Layers
    'Wd1': weight_variable((5*5*16,120)),
    'Wd2': weight_variable((120,84)),
    'Wd3': weight_variable((84,10))
    }
                           
    biases = {
    # Convolution Layers
    'bc1': bias_variable((1,6)), #size = 1 ?
    'bc2': bias_variable((1,16)) ,
    
    # Dense Layers
    'bd1': bias_variable((1,120)),
    'bd2': bias_variable((1,84)),
    'bd3': bias_variable((1,10))
    }
        
    #layer 1
    conv1  = conv2d(image, weights['Wc1'], biases['bc1']) 
    pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 
        #layer1  =tf.nn.conv2d( image, tf.constant([filter_height, filter_width, in_channels, out_channels]), strides, padding, data_format='NHWC', dilations=None, name=None)
        #model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=shape))
        #model.add(layers.MaxPooling2D((2, 2)))
    
    #layer 2
    conv2  = conv2d(pool1, weights['Wc2'], biases['bc2'], strides=1, padding_ = 'VALID') 
    pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 
    flattened = tf.reshape(pool2, [-1, 5*5*16]) 
        # model.add(layers.Conv2D(16, (5, 5), activation='relu',  padding='same', input_shape=(14,14,6)))
        # model.add(layers.MaxPooling2D((2, 2)))
        #model.add(layers.Flatten())
        # model.add(layers.Reshape((5*5*16, 1), input_shape=(5*5*16,)))
    
    #layer 3
    dense1 =  tf.nn.relu(tf.add(tf.matmul(flattened, weights['Wd1']), biases['bd1']))
        #model.add(layers.Dense(120, activation='relu')) #, input_shape=(10, 10)))
    
    #layer 4
    dense2 =  tf.nn.relu(tf.add(tf.matmul(dense1, weights['Wd2']), biases['bd2']))
        #model.add(layers.Dense(84, activation='relu'))
    
    #layer 5
    pred = tf.nn.softmax(tf.matmul(dense2,  weights['Wd3']) + biases['bd3'])
        #model.add(layers.Dense(10, activation='relu'))         
        #pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
        
    return pred
                         
 #-------------------- KERAS ---------------------------------------   
   # model = models.Sequential()
    
    #layer 1
    #model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=shape))
    #model.add(layers.MaxPooling2D((2, 2)))
    
    #layer 2
   # model.add(layers.Conv2D(16, (5, 5), activation='relu',  padding='same', input_shape=(14,14,6)))
   # model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Flatten())
   # model.add(layers.Reshape((5*5*16, 1), input_shape=(5*5*16,)))
    
    #layer 3
    #model.add(layers.Dense(120, activation='relu')) #, input_shape=(10, 10)))
    
    #layer 4
    #model.add(layers.Dense(84, activation='relu'))
    
    #layer 5
    #model.add(layers.Dense(10, activation='relu')) 
  #----------------------------TENSORFLOW CORE --------------------------------------


<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

 Your answer goes here in details 

     My answer: nbr_parameters = (5x5)x6 + 16x(5x5) + 16x25x120 + 120x84 + 84x10 = 59470 parameters to train 
     (it could have been far worse if he had not used convolutions...)

<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [6]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'log_files/'

# Model, loss function and accuracy
# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 28,28,1], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
#W = tf.Variable(tf.zeros([784, 10]), name='Weights')
#b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = LeNet5_Model(x)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32)) 


<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [7]:
def evaluate(logits, labels, sess):
    # logits will be the outputs of your model, labels will be one-hot vectors corresponding to the actual labels
    # logits and labels are numpy arrays
    # this function should return the accuracy of your model
    accuracy = sess.run([acc], feed_dict = {pred : logits, y : labels})
    
    return accuracy

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [8]:
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_LeNet-5_SGD", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

import time #to check the time


# Initializing the variables
def train(init, sess, logs_path, n_epochs, batch_size, optimizer, cost, merged_summary_op):
    # optimizer and cost are the same kinds of objects as in Section 1
    # Train your model
    start_time = time.time()
    sess.run(init)
    
    display_step = 4
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(n_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            batch_xs = batch_xs.reshape((batch_xs.shape[0],28,28,1))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()
    
    
    # Print the accuracy on testing data
    labels = mnist.test.labels
    input_test = mnist.test.images.reshape((mnist.test.images.shape[0],28,28,1))
    logits = np.array(sess.run([pred], feed_dict={x:input_test}))
    logits = logits.reshape((logits.shape[1],10))
    accuracy = evaluate(logits,labels, sess)
    print("Accuracy:"+ str(accuracy[0]))
    print("--- %s seconds ---" % (time.time() - start_time))
    #print("Accuracy:"+ acc.eval({x: mnist.test.images, y: mnist.test.labels}, session = sess))
        
        
with tf.Session() as sess:
    #train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)
    train(init, sess, logs_path, 10, batch_size, optimizer, cost, merged_summary_op)
    

Epoch:  04   =====> Loss= 2.134733905
Epoch:  08   =====> Loss= 0.586842939
Optimization Finished!
Accuracy:0.8974
--- 167.30340194702148 seconds ---


<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

Please put your loss and accuracy curves here.

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |         0.9525        |        ???          |       
| Training Time        |         677 s        |        ???          |  

- Which optimizer gives the best accuracy on test data?

**Your answer:** ...


In [9]:
tf.reset_default_graph()
# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'log_files/'

# Model, loss function and accuracy
# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 28,28,1], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
#W = tf.Variable(tf.zeros([784, 10]), name='Weights')
#b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = LeNet5_Model(x)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
  
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32)) 
# your implementation goes here
with tf.name_scope('AdamGD'):
    # Gradient Descent
    AdamOptimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_LeNet-5_SGD", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess3:
    #train(init, sess, logs_path, training_epochs, batch_size, AdamOptimizer, cost, merged_summary_op)
    train(init, sess3, logs_path, 1, batch_size, AdamOptimizer, cost, merged_summary_op)

Optimization Finished!
Accuracy:0.9724
--- 18.180661916732788 seconds ---


<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** ...

In [10]:

def LeNet5_Model_Dropout(image):    
    # your implementation goes here
    shape = image.shape
    
    
    weights = {
    # Convolution Layers
    'Wc1': weight_variable((5,5,1,6)), 
    'Wc2': weight_variable((5,5,6,16)) ,
    
    # Dense Layers
    'Wd1': weight_variable((5*5*16,120)),
    'Wd2': weight_variable((120,84)),
    'Wd3': weight_variable((84,10))
    }
                           
    biases = {
    # Convolution Layers
    'bc1': bias_variable((1,6)), #size = 1 ?
    'bc2': bias_variable((1,16)) ,
    
    # Dense Layers
    'bd1': bias_variable((1,120)),
    'bd2': bias_variable((1,84)),
    'bd3': bias_variable((1,10))
    }
        
    #layer 1
    conv1  = conv2d(image, weights['Wc1'], biases['bc1']) 
    pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 
  
    #layer 2
    conv2  = conv2d(pool1, weights['Wc2'], biases['bc2'], strides=1, padding_ = 'VALID') 
    pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 
    flattened = tf.reshape(pool2, [-1, 5*5*16]) 
    #let's add dropout
    dropout = tf.nn.dropout(flattened, keep_prob = 0.75)

    #layer 3
    dense1 =  tf.nn.relu(tf.add(tf.matmul(dropout, weights['Wd1']), biases['bd1']))
  
    #layer 4
    dense2 =  tf.nn.relu(tf.add(tf.matmul(dense1, weights['Wd2']), biases['bd2']))

    #layer 5
    pred = tf.nn.softmax(tf.matmul(dense2,  weights['Wd3']) + biases['bd3'])
  
    return pred

tf.reset_default_graph()
# your implementation goes here
# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'log_files/'

# Model, loss function and accuracy
# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 28,28,1], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
#W = tf.Variable(tf.zeros([784, 10]), name='Weights')
#b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = LeNet5_Model_Dropout(x)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
with tf.name_scope('AdamGD'):
    # Gradient Descent
    AdamOptimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32)) 
  
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_LeNet-5_SGD", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    #train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)
     train(init, sess, logs_path, 1, batch_size, AdamOptimizer, cost, merged_summary_op)


Optimization Finished!
Accuracy:0.9656
--- 18.512969493865967 seconds ---
